In [ ]:
import numpy as np
import cv2
import rosbag
from cv_bridge import CvBridge
from matplotlib import pyplot as plt
import ros_numpy
from tf.transformations import quaternion_matrix
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

Load data (camera image and pointcloud) and extract one frame

In [ ]:
# Load rosbag
bag = rosbag.Bag('/home/user/rosbags/2021-12-03-20-27-23.bag')
frame = 100

# ZED camera
i = 0
for topic, msg, t in bag.read_messages(topics='/zed2i/zed_node/left/image_rect_color'):
    img = msg
    i+=1
    if i == frame:
        break
    
# Velodyne point cloud
i = 0
for topic, msg, t in bag.read_messages(topics='/velodyne_points'):
    pc = msg
    i+=1
    if i == frame:
        break

Convert msgs to numpy array

In [ ]:
# Convert from image msg to numpy array
bridge = CvBridge()
img = bridge.imgmsg_to_cv2(img, desired_encoding='passthrough')

# Convert point cloud msg to numpy array
pc_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pc, remove_nans=True)

# Display image frame
px.imshow(img)

Get transformation from lidar to camera frame:

1) Add `static_transform_publisher` with zed and lidar transforms to base_link to a launch file and launch it
2) Run `get_tfs.py` which returns the translation `(x, y, z)` and rotation `(qx, qy, qz, qw)` between two frames
3) Copy them from terminal
4) Insert them below

In [ ]:
# Transform lidar to camera left optical (using tf node with lookupTransform)
trans = [0.06500000000000006, -0.029994648094052723, -0.4211903145677902]
quat = [0.45451945741323646, 0.45451945741323657, -0.5416752374188588, 0.5416752374188588]

# Convert quaternion to rotation matrix
rotMat = quaternion_matrix(quat)[:3, :3]
# Convert rotation matrix to rotation vector
rot_vec, _ = cv2.Rodrigues(rotMat)

Get camera parameters

1) Start rosbag with zed recording and echo `camera_info` topic
2) Copy important information
    1) D are the distortion parameters (**needed**)
    2) K is the intrinsic camera matrix (for the raw (distorted) images) (**needed**)
    3) R is the Rectification matrix 
    4) P is the projection/camera matrix
3) Insert them below

In [ ]:
# Lidar points
#! Next code only because lidar was facing backwards
# Remove points which face opposite direction of camera
# < 0 because lidar was mounted the wrong way (facing backwards)
pc_array = pc_array[pc_array[:,0] < 0]
# Remove points behind car (because not visible in camera frame)
# pc_array = pc_array[pc_array[:,0] > 0]
objectPoints = pc_array

# Rotation vector from lidar frame to camera frame
rvec = rot_vec

# Translation vector from lidar to camera frame
tvec = np.asarray(trans)

# Camera matrix (K)
cameraMatrix = np.array(
    [[519.9877319335938, 0.0, 630.9716796875], 
    [0.0, 519.9877319335938, 352.2940673828125], 
    [0.0, 0.0, 1.0]])

# Distortion coefficients (D)
distCoeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

Project 3D lidar points onto 2D plane

In [ ]:
# Projection of 3D points onto 2D plane
lidar_img_points, _ = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
lidar_img_points = lidar_img_points.reshape(len(lidar_img_points), 2)

# Remove points which do not lie inside image plane
lidar_img_points_small = lidar_img_points[
    (lidar_img_points[:,0] > 0)
    & (lidar_img_points[:,0] < img.shape[1])
    & (lidar_img_points[:,1] > 0)
    & (lidar_img_points[:,1] < img.shape[0])
    ]

# Same for original pointcloud
pc_array_small = pc_array[
    (lidar_img_points[:,0] > 0)
    & (lidar_img_points[:,0] < img.shape[1])
    & (lidar_img_points[:,1] > 0)
    & (lidar_img_points[:,1] < img.shape[0])
    ]

# Create dataframe with x, y coordinates and distance of lidar points 
df = pd.DataFrame(lidar_img_points_small, columns=['x', 'y'])
# Add distance information
#! - because lidar was facing backward
df['dist'] = -pc_array_small[:,0]
df.head()

Visualize

In [ ]:
fig = px.imshow(img)
# fig.add_trace(px.scatter(lidar_img_points_small[:, 0], lidar_img_points_small[:, 1]))
fig.add_trace(go.Scatter(
    x=df['x'], y=df['y'],
    mode='markers', 
    marker=dict(size=3, opacity=0.8, color=df['dist'])))
fig.update_layout(width = 1920*0.7, height=1080*0.7)
fig.show()